In [1]:
#from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
from torch import optim
import torch.nn as nn
import pandas as pd
import numpy as np
import time
import torch
from node2vec import Node2Vec
node2vec = Node2Vec(graph_train)
model = node2vec.fit()

In [8]:
import csv
import numpy as np
from sklearn.metrics import accuracy_score, log_loss
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression


# Read sequences
sequences = list()
with open('sequences.txt', 'r') as f:
    for line in f:
        sequences.append(line[:-1])

# Split data into training and test sets
sequences_train = list()
sequences_test = list()
proteins_test = list()
y_train = list()
with open('graph_labels.txt', 'r') as f:
    for i,line in enumerate(f):
        t = line.split(',')
        if len(t[1][:-1]) == 0:
            proteins_test.append(t[0])
            sequences_test.append(sequences[i])
        else:
            sequences_train.append(sequences[i])
            y_train.append(int(t[1][:-1]))

# Map sequences to 
vec = TfidfVectorizer(analyzer='char', ngram_range=(1, 3))
X_train = vec.fit_transform(sequences_train)
X_test = vec.transform(sequences_test)

  (0, 2659)	0.06149261120321289
  (0, 5624)	0.06066983048295409
  (0, 3661)	0.06453838750552988
  (0, 8228)	0.05702822888198667
  (0, 8454)	0.05929196233215155
  (0, 5911)	0.06528939114813927
  (0, 295)	0.057127014497987655
  (0, 5927)	0.05303624546194652
  (0, 297)	0.042408878390136025
  (0, 7196)	0.045462188676295645
  (0, 7539)	0.039399051440049786
  (0, 1219)	0.04217369121444537
  (0, 4706)	0.05071772169400806
  (0, 235)	0.05241401649487533
  (0, 8058)	0.05495239375396609
  (0, 1244)	0.05125736549910664
  (0, 906)	0.054869681864821065
  (0, 2579)	0.06588462904907086
  (0, 6887)	0.06066983048295409
  (0, 7101)	0.06040745570576998
  (0, 3077)	0.04704777226001155
  (0, 997)	0.04847387099204874
  (0, 4274)	0.06192579127271886
  (0, 6127)	0.06284121730862169
  (0, 307)	0.05965257844607174
  :	:
  (4887, 2997)	0.007969595601046764
  (4887, 6061)	0.01637994567904733
  (4887, 4941)	0.019201285801914113
  (4887, 7412)	0.016182409968264116
  (4887, 6250)	0.008848015983356461
  (4887, 1688)	0

In [11]:
print(len(y_train))
print(X_train.shape)

4888
(4888, 8466)


In [2]:
# Get data

y_train = pd.read_csv('sample_submission_test.csv').iloc[: , 1:].to_numpy()
X_train_seq = pd.read_csv('sample_submission_seq.csv').iloc[: , 1:].to_numpy()
X_train_str = pd.read_csv('sample_submission_str.csv').iloc[: , 1:].to_numpy()

print(X_train_seq, np.shape(X_train_str), np.shape(y_train))

[[0.01342103 0.01012509 0.46634627 ... 0.07674363 0.0174905  0.00991321]
 [0.02174239 0.01612491 0.41744692 ... 0.06925897 0.01545664 0.00779209]
 [0.05807513 0.01203749 0.2246163  ... 0.04407045 0.01212762 0.005458  ]
 ...
 [0.02276303 0.01242107 0.24162626 ... 0.04877692 0.01632783 0.00632687]
 [0.02810728 0.01194174 0.059683   ... 0.04407273 0.01484368 0.02052648]
 [0.13940033 0.00669199 0.01607577 ... 0.01033002 0.00609641 0.00639728]] (1223, 18) (1223, 18)


In [3]:
input_dim = 18
hidden_dim = 1024
dropout = 0.2
n_class = 18
epochs = 50
batch_size = 64
learning_rate = 0.001
N_train = X_train_seq.shape[0]
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [4]:
class Ensemble_Baseline(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout, n_class):
        super(Ensemble_Baseline, self).__init__()
        self.fc1 = nn.Linear(2*input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim//2)
        self.fc3 = nn.Linear(hidden_dim//2, n_class)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, x_seq, x_str):
        x = torch.cat((x_seq, x_str), dim=1)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)

        x = self.relu(self.fc2(x))    
        x = self.dropout(x)
        out = self.fc3(x)

        return F.log_softmax(out, dim=1)


In [5]:
# Initializes model and optimizer
model = Ensemble_Baseline(input_dim, hidden_dim, dropout, n_class).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

In [6]:
# Train model
#tb = SummaryWriter()
for epoch in range(epochs):
    t = time.time()
    model.train()
    train_loss = 0
    correct = 0
    count = 0
    # Iterate over the batches
    for i in range(0, 1223, batch_size):

        x_seq_batch = torch.FloatTensor(X_train_seq[i:min(i+batch_size, 1223)]).to(device)
        x_str_batch = torch.FloatTensor(X_train_str[i:min(i+batch_size, 1223)]).to(device)
        y_batch = torch.FloatTensor(y_train[i:min(i+batch_size, 1223)]).to(device)
        
        optimizer.zero_grad()
        output = model(x_seq_batch, x_str_batch)
        loss = loss_function(output, y_batch)
        train_loss += loss.item() * output.size(0)
        count += output.size(0)
        preds = output.max(1)[1].type_as(y_batch)

        loss.backward()
        optimizer.step()
            
    # Tensorboard
#     tb.add_scalar('loss_train: {:.4f}'.format(train_loss / count), epoch+1)
#     tb.add_scalar('acc_train: {:.4f}'.format(correct / count), epoch+1)
    
    if epoch % 5 == 0:
        print('Epoch: {:03d}'.format(epoch+1),
              'loss_train: {:.4f}'.format(train_loss / count),
              'time: {:.4f}s'.format(time.time() - t))

        
# tb.flush()
# tb.close()

        
#run tensorboard --logdir=runs

Epoch: 001 loss_train: 2.4782 time: 0.9118s
Epoch: 006 loss_train: 2.1346 time: 0.0350s
Epoch: 011 loss_train: 2.1236 time: 0.0347s
Epoch: 016 loss_train: 2.1188 time: 0.0347s
Epoch: 021 loss_train: 2.1160 time: 0.0347s
Epoch: 026 loss_train: 2.1141 time: 0.0347s
Epoch: 031 loss_train: 2.1126 time: 0.0346s
Epoch: 036 loss_train: 2.1116 time: 0.0344s
Epoch: 041 loss_train: 2.1106 time: 0.0323s
Epoch: 046 loss_train: 2.1106 time: 0.0322s


In [7]:
model.eval()
with torch.no_grad():
    dataloader = DataLoader(dataset=test_ohe,
                            batch_size=1,
                            shuffle=True,
                            pin_memory=True,
                            drop_last=True,
                            )
    y_pred = torch.zeros(1,18).to(device)
    for idx, data in enumerate(dataloader):
        y_pred = torch.cat([y_pred, model(data.to(device))[0]], 0)
    y_pred = y_pred[1:]


NameError: name 'DataLoader' is not defined

In [ ]:
print(y_pred)

In [ ]:
import csv
# Write predictions to a file
with open('sample_submission_han.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    lst = list()
    for i in range(18):
        lst.append('class'+str(i))
    lst.insert(0, "name")
    writer.writerow(lst)
    for i, protein in enumerate(proteins_test):
        lst = y_pred[i].tolist()
        lst.insert(0, protein)
        writer.writerow(lst)